## Statisitically Testing 2P Responses to Stimulus

### Environment Setup

In [ ]:
### if running on Google Colab, run this cell once, then restart the runtime and run the rest of the notebook
import os
if "COLAB_GPU" in os.environ:
    !wget https://raw.githubusercontent.com/AllenInstitute/openscope_databook/main/requirements.txt
    %pip install -r requirements.txt --user

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np

from dandi import dandiapi
from pynwb import NWBHDF5IO
from scipy import interpolate

%matplotlib inline

### Downloading Ophys File
Change the values below to download the file you’re interested in. Set `dandiset_id` and `dandi_filepath` to correspond to the dandiset id and filepath of the file you want. If you’re accessing an embargoed dataset, change `authenticate` to True, and set `dandi_api_key` to your DANDI API key. If you want to stream a file instead of downloading it, checkout Streaming an NWB File with fsspec.

In [ ]:
dandiset_id = "000336"
dandi_filepath = "sub_1170622509/sub_1170622509sess_1194683535/sub_1170622509+sess_1194683535+exp_1194752582_ophys.nwb"
authenticate = True
dandi_api_key = os.environ["DANDI_API_KEY"]
download_loc = "."

In [ ]:
filename = dandi_filepath.split("/")[-1]
filepath = f"{download_loc}/{filename}"

In [ ]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)
print(f"Got dandiset {my_dandiset}")

In [ ]:
file = my_dandiset.get_asset_by_path(dandi_filepath)
# this may take awhile, especially if the file to download is large
file.download(filepath)

print(f"Downloaded file to {filepath}")

### Extracting 2P Data and Stimulus Data
Below, the fluorescence traces and timestamps are read from the file's `Processing` section. Note that the exact format to access these traces can vary between newer and older NWB files, so some adjustments may be necessary. Additionally, the stimulus data is also read from the NWB file's `Intervals` section. Stimulus information is stored as a series of tables depending on the type of stimulus shown in the session. One such table is displayed below, as well as the names of all the tables.

In [ ]:
io = NWBHDF5IO(filepath, mode="r", load_namespaces=True)
nwb = io.read() 

In [ ]:
dff = nwb.processing["ophys"]["dff"]
dff_trace = dff.roi_response_series["traces"].data
dff_timestamps = dff.roi_response_series["traces"].timestamps

flr = nwb.processing["ophys"]["corrected_fluorescence"]
flr_trace = flr.roi_response_series["traces"].data
flr_timestamps = flr.roi_response_series["traces"].timestamps

# accessing the above data may look different for older nwb files, like the following
# dff_trace = dff.roi_response_series["RoiResponseSeries"].data
# dff_timestamps = dff.roi_response_series["RoiResponseSeries"].timestamps

print(flr_trace.shape)
print(flr_timestamps.shape)
print(dff_trace.shape)
print(dff_timestamps.shape)

In [ ]:
stimulus_names = list(nwb.intervals.keys())
print(stimulus_names)

In [ ]:
stim_table = nwb.intervals['gratings_presentations']
print(stim_table.colnames)
stim_table[:10]